In [2]:
import numpy as np

def split_sequence(sequence, sliding_window_width):
    X, y = [], []
    for i in range(len(sequence)):
        # 找到最后一次滑动所截取数据中最后一个元素的索引，
        # 如果这个索引超过原序列中元素的索引则不截取；
        end_element_index = i + sliding_window_width
        if end_element_index > len(sequence) - 1: # 序列中最后一个元素的索引
            break
        sequence_x, sequence_y = sequence[i:end_element_index], sequence[end_element_index] # 取最后一个元素作为预测值y
        X.append(sequence_x)
        y.append(sequence_y)
    
    #return X,y
    return np.array(X), np.array(y)

if __name__ == '__main__':
    seq_test = [10,20,30,40,50,60,70,80,90]
    sw_width = 3
    seq_test_x, seq_test_y = split_sequence(seq_test, sw_width)
    print(seq_test_x.shape,seq_test_y.shape)
    for i in zip(seq_test_x,seq_test_y):
        print(i)
    for i in range(len(seq_test_x)):
        print(seq_test_x[i], seq_test_y[i])

(6, 3) (6,)
(array([10, 20, 30]), 40)
(array([20, 30, 40]), 50)
(array([30, 40, 50]), 60)
(array([40, 50, 60]), 70)
(array([50, 60, 70]), 80)
(array([60, 70, 80]), 90)
[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=sw_width))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [5]:
model.fit(seq_test_x, seq_test_y, epochs=2000, verbose=0)

In [7]:
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, sw_width))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[101.17381]]


In [8]:
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [9]:
out_seq

array([ 25,  45,  65,  85, 105, 125, 145, 165, 185])

In [10]:
len(out_seq)

9

In [11]:
# 每一个数组先转换成9×1的二维数组
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# 使用numpy的hstack方法，沿水平方向堆叠数组，对于二维数组就是沿第二个维度（列）堆叠
dataset = np.hstack((in_seq1, in_seq2, out_seq))

In [12]:
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [18]:
def split_sequences(sequences, sliding_window_width):
    X, y = [], []
    for i in range(len(sequences)):
        # 找到最后一次滑动所截取数据中最后一个元素的索引，
        # 如果这个索引超过原序列中元素的索引则不截取；
        end_element_index = i + sliding_window_width
        if end_element_index > len(sequences) : # 序列中最后一个元素的索引
            break
        
        # 使用二维数组切片来截取输入数据X和标签y；:-1 表示截取第1,2列数据（共3列）；-1表示截取最后一列数据；
        sequence_x, sequence_y = sequences[i:end_element_index ,:-1], sequences[end_element_index-1, -1] # 取最后一个元素作为预测值y
        X.append(sequence_x)
        y.append(sequence_y)
    
    #return X,y
    return np.array(X), np.array(y)

if __name__ == '__main__':
    sw_width = 3
    X, y = split_sequences(dataset, sw_width)
    print(X.shape, y.shape)
    for i in range(len(X)):
        print(X[i], y[i])

(7, 3, 2) (7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [19]:
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))

In [20]:
n_input

6

In [21]:
X

array([[10, 15, 20, 25, 30, 35],
       [20, 25, 30, 35, 40, 45],
       [30, 35, 40, 45, 50, 55],
       [40, 45, 50, 55, 60, 65],
       [50, 55, 60, 65, 70, 75],
       [60, 65, 70, 75, 80, 85],
       [70, 75, 80, 85, 90, 95]])

In [28]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X, y, epochs=2000, verbose=0)

In [34]:
x_input = np.array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)

In [35]:
x_input

array([[ 80,  85,  90,  95, 100, 105]])

In [36]:
print(yhat)

[[206.24257]]


In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,concatenate
from tensorflow.keras.utils import plot_model
import numpy as np

def split_sequences(sequences, sliding_window_width):
    X, y = [], []
    for i in range(len(sequences)):
        # 找到最后一次滑动所截取数据中最后一个元素的索引，
        # 如果这个索引超过原序列中元素的索引则不截取；
        end_element_index = i + sliding_window_width
        if end_element_index > len(sequences) : # 序列中最后一个元素的索引
            break
        
        # 使用二维数组切片来截取输入数据X和标签y；:-1 表示截取第1,2列数据（共3列）；-1表示截取最后一列数据；
        sequence_x, sequence_y = sequences[i:end_element_index ,:-1], sequences[end_element_index-1, -1] # 取最后一个元素作为预测值y
        X.append(sequence_x)
        y.append(sequence_y)
    
    #return X,y
    return np.array(X), np.array(y)

def multiple_input_series(X_1, X_2, y, sliding_window_width, epochs_num):
    visible1 = Input(shape=(sliding_window_width,))
    dense1 = Dense(100, activation='relu')(visible1)
    
    visible2 = Input(shape=(sliding_window_width,))
    dense2 = Dense(100, activation='relu')(visible2)
    
    # 构建多输入多输出模型
    merge = concatenate([dense1, dense2])
    output = Dense(1)(merge)
    print('output:',output)

    model = Model(inputs=[visible1, visible2], outputs=output)
    model.compile(optimizer='adam', loss='mse')
    
    plot_model(model, to_file='mis_model.png', show_shapes=True, show_layer_names=True, rankdir='TB', dpi=200)

    model.fit([X_1, X_2], y, epochs=epochs_num, verbose=0)
    
    return model

if __name__ == '__main__':
    sw_width = 3
    epochs_num = 2000
    
    # 训练数据
    in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
    in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
    out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    out_seq = out_seq.reshape((len(out_seq), 1))

    dataset = np.hstack((in_seq1, in_seq2, out_seq))
    
    # 训练数据和标签划分
    X, y = split_sequences(dataset, sw_width)
    
    print(X.shape, y.shape)
#     for i in range(len(X)):
#         print(X[i], y[i])
    
    X1 = X[:, :, 0]
    X2 = X[:, :, 1]
    
    # 训练模型
    model = multiple_input_series(X1, X2, y, sw_width, epochs_num)
    
    # 构造测试数据
    x_test = np.array([[80, 85], [90, 95], [100, 105]])
    # 将测试数据重塑为二维数组
    x1 = x_test[:, 0].reshape((1, sw_width))
    x2 = x_test[:, 1].reshape((1, sw_width))
    # 模型预测
    yhat = model.predict([x1, x2], verbose=0)
    print(yhat)


(7, 3, 2) (7,)
output: Tensor("dense_14/Identity:0", shape=(None, 1), dtype=float32)
[[206.31477]]


In [8]:
x_test.shape

(3, 2)

In [10]:
x1.shape

(1, 3)

In [12]:
X1.shape

(7, 3)

In [13]:
in_seq1.shape

(9, 1)

In [15]:
out_seq.shape

(9, 1)

In [14]:
dataset.shape

(9, 3)

In [16]:
X.shape

(7, 3, 2)

###################################################################

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

def split_sequences(sequence, sliding_window_width):
    X, y = [], []
    for i in range(len(sequence)):
        # 找到最后一次滑动所截取数据中最后一个元素的索引，
        # 如果这个索引超过原序列中元素的索引则不截取；
        end_element_index = i + sliding_window_width
        if end_element_index > len(sequence) - 1: # 序列中最后一个元素的索引
            break
        
        sequence_x, sequence_y = sequence[i:end_element_index], sequence[end_element_index, :] # 取最后一列元素作为预测值y
        X.append(sequence_x)
        y.append(sequence_y)
    
    #return X,y
    return np.array(X), np.array(y)

def vector_output_model(n_input, n_output, epochs_num):
    model = Sequential()
    model.add(Dense(100, activation='relu', input_dim=n_input))
    model.add(Dense(n_output))
    model.compile(optimizer='adam', loss='mse')

    model.fit(X, y, epochs=epochs_num, verbose=0)
    
    return model

if __name__ == '__main__':
    sw_width = 3
    epochs_num = 2000
    
    # 训练数据
    in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
    in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
    out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    out_seq = out_seq.reshape((len(out_seq), 1))

    dataset = np.hstack((in_seq1, in_seq2, out_seq))
    
    # 训练数据和标签划分
    X, y = split_sequences(dataset, sw_width)
    
    print(X.shape, y.shape)
#     for i in range(len(X)):
#         print(X[i], y[i])
        
    n_input = X.shape[1] * X.shape[2]
    X = X.reshape((X.shape[0], n_input))
    n_output = y.shape[1]

    x_input = np.array([[70,75,145], [80,85,165], [90,95,185]])
    x_input = x_input.reshape((1, n_input))
    
    model = vector_output_model(n_input, n_output, epochs_num)
    
    yhat = model.predict(x_input, verbose=0)
    print(yhat)

(6, 3, 3) (6, 3)
[[101.39276  105.340294 207.98416 ]]


##########################################

In [24]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import plot_model
import numpy as np

def split_sequences(sequence, sliding_window_width):
    X, y = [], []
    for i in range(len(sequence)):
        # 找到最后一次滑动所截取数据中最后一个元素的索引，
        # 如果这个索引超过原序列中元素的索引则不截取；
        end_element_index = i + sliding_window_width
        if end_element_index > len(sequence) - 1: # 序列中最后一个元素的索引
            break
        
        sequence_x, sequence_y = sequence[i:end_element_index, :], sequence[end_element_index, :] # 取最后一列元素作为预测值y
        X.append(sequence_x)
        y.append(sequence_y)
    
    #return X,y
    return np.array(X), np.array(y)

def multi_output_model(n_input, y, epochs_num):
    visible = Input(shape=(n_input,))
    dense = Dense(100, activation='relu')(visible)

    output1 = Dense(1)(dense)
    output2 = Dense(1)(dense)
    output3 = Dense(1)(dense)

    model = Model(inputs=visible, outputs=[output1, output2, output3])
    model.compile(optimizer='adam', loss='mse')
    
    plot_model(model, to_file='multi_output_model.png', show_shapes=True, show_layer_names=True, rankdir='TB', dpi=200)

    model.fit(X, y, epochs=2000, verbose=0)
    
    return model

if __name__ == '__main__':
    sw_width = 3
    epochs_num = 2000
    
    # 训练数据
    in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
    in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
    out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    out_seq = out_seq.reshape((len(out_seq), 1))

    dataset = np.hstack((in_seq1, in_seq2, out_seq))
    
    # 训练数据和标签划分
    X, y = split_sequences(dataset, sw_width)
    
    print(X.shape, y.shape)
#     for i in range(len(X)):
#         print(X[i], y[i])
        
    n_input = X.shape[1] * X.shape[2]
    X = X.reshape((X.shape[0], n_input))

    y1 = y[:, 0].reshape((y.shape[0], 1))
    y2 = y[:, 1].reshape((y.shape[0], 1))
    y3 = y[:, 2].reshape((y.shape[0], 1))         
    y_list = [y1,y2,y3]
    
    model = multi_output_model(n_input, y_list, epochs_num)
    
    x_input = np.array([[70,75,145], [80,85,165], [90,95,185]])
    x_input = x_input.reshape((1, n_input))
    yhat = model.predict(x_input, verbose=0)
    print(yhat)

(6, 3, 3) (6, 3)
[array([[100.50821]], dtype=float32), array([[105.74859]], dtype=float32), array([[206.86055]], dtype=float32)]


In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

def split_sequence(sequence, m_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(sequence)):

        end_element_index = i + n_steps_in
        out_end_index = end_element_index + n_steps_out
        if out_end_index > len(sequence): 
            break
        
        sequence_x, sequence_y = sequence[i:end_element_index], sequence[end_element_index:out_end_index]
        X.append(sequence_x)
        y.append(sequence_y)

    return np.array(X), np.array(y)

def vector_output_model(n_steps_in, n_steps_out, X, y, epochs_num):
    model = Sequential()
    model.add(Dense(100, activation='relu', input_dim=n_steps_in))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mse')
    
    model.fit(X, y, epochs=2000, verbose=0)
    return model

if __name__ == '__main__':
    epochs_num = 2000
    n_steps_in, n_steps_out = 3, 2
    
    raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
    
    print(X.shape, y.shape)
    for i in range(len(X)):
        print(X[i], y[i])
    
    model = vector_output_model(n_steps_in, n_steps_out, X, y, epochs_num)
    
    x_input = np.array([70, 80, 90])
    x_input = x_input.reshape((1, n_steps_in))
    yhat = model.predict(x_input, verbose=0)
    print(yhat)

(5, 3) (5, 2)
[10 20 30] [40 50]
[20 30 40] [50 60]
[30 40 50] [60 70]
[40 50 60] [70 80]
[50 60 70] [80 90]
[[100.4229  111.65523]]


In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(sequences)):

        end_element_index = i + n_steps_in
        out_end_index = end_element_index + n_steps_out - 1
        
        if out_end_index > len(sequences): 
            break
        
        sequence_x, sequence_y = sequences[i:end_element_index,:-1], sequences[end_element_index-1:out_end_index,-1]
        X.append(sequence_x)
        y.append(sequence_y)

    return np.array(X), np.array(y)

def multi_step_output_model(n_input, n_steps_out, X, y, epochs_num):
    model = Sequential()
    model.add(Dense(100, activation='relu', input_dim=n_input))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mse')
    
    model.fit(X, y, epochs=epochs_num, verbose=0)
    return model

if __name__ == '__main__':
    epochs_num = 2000
    
    in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
    in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
    out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    out_seq = out_seq.reshape((len(out_seq), 1))

    dataset = np.hstack((in_seq1, in_seq2, out_seq))

    n_steps_in, n_steps_out = 3, 2

    X, y = split_sequences(dataset, n_steps_in, n_steps_out)

    n_input = X.shape[1] * X.shape[2]
    X = X.reshape((X.shape[0], n_input))
    
    print(X.shape, y.shape)
    for i in range(len(X)):
        print(X[i], y[i])
    
    model = multi_step_output_model(n_input, n_steps_out, X, y, epochs_num)
    
    x_input = np.array([[70, 75], [80, 85], [90, 95]])
    x_input = x_input.reshape((1, n_input))
    yhat = model.predict(x_input, verbose=0)
    print(yhat)

(6, 6) (6, 2)
[10 15 20 25 30 35] [65 85]
[20 25 30 35 40 45] [ 85 105]
[30 35 40 45 50 55] [105 125]
[40 45 50 55 60 65] [125 145]
[50 55 60 65 70 75] [145 165]
[60 65 70 75 80 85] [165 185]
[[186.31998 206.29776]]


In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(sequences)):

        end_element_index = i + n_steps_in
        out_end_index = end_element_index + n_steps_out
        
        if out_end_index > len(sequences): 
            break
        
        sequence_x, sequence_y = sequences[i:end_element_index,:], sequences[end_element_index:out_end_index,:]
        X.append(sequence_x)
        y.append(sequence_y)

    return np.array(X), np.array(y)

def multi_parallel_output_model(n_input, n_output, X, y, epochs_num):
    model = Sequential()
    model.add(Dense(100, activation='relu', input_dim=n_input))
    model.add(Dense(n_output))
    model.compile(optimizer='adam', loss='mse')
    
    model.fit(X, y, epochs=epochs_num, verbose=0)
    return model

if __name__ == '__main__':
    epochs_num = 2000
    
    in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
    in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
    out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    out_seq = out_seq.reshape((len(out_seq), 1))
    # 沿列堆叠数组，相当于列数增加
    dataset = np.hstack((in_seq1, in_seq2, out_seq))
    # 时间步长
    n_steps_in, n_steps_out = 3, 2
    # 将输入数据分为训练数据和训练标签
    X, y = split_sequences(dataset, n_steps_in, n_steps_out)
    # 展平输入数据
    n_input = X.shape[1] * X.shape[2]
    X = X.reshape((X.shape[0], n_input))
    # 展平输出数据
    n_output = y.shape[1] * y.shape[2]
    y = y.reshape((y.shape[0], n_output))
    
    print(X.shape, y.shape)
    for i in range(len(X)):
        print(X[i], y[i])
    
    model = multi_parallel_output_model(n_input, n_output, X, y, epochs_num)
    
    x_input = np.array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
    x_input = x_input.reshape((1, n_input))
    yhat = model.predict(x_input, verbose=0)
    print(yhat)

(5, 9) (5, 6)
[10 15 25 20 25 45 30 35 65] [ 40  45  85  50  55 105]
[20 25 45 30 35 65 40 45 85] [ 50  55 105  60  65 125]
[ 30  35  65  40  45  85  50  55 105] [ 60  65 125  70  75 145]
[ 40  45  85  50  55 105  60  65 125] [ 70  75 145  80  85 165]
[ 50  55 105  60  65 125  70  75 145] [ 80  85 165  90  95 185]
[[ 92.22672   96.749825 188.34888  102.91172  108.4422   209.61449 ]]


(6, 9) (6, 3)
[10 15 25 20 25 45 30 35 65] [40 45 85]
[20 25 45 30 35 65 40 45 85] [ 50  55 105]
[ 30  35  65  40  45  85  50  55 105] [ 60  65 125]
[ 40  45  85  50  55 105  60  65 125] [ 70  75 145]
[ 50  55 105  60  65 125  70  75 145] [ 80  85 165]
[ 60  65 125  70  75 145  80  85 165] [ 90  95 185]
[[ 89.99398  95.03327 185.03163]]